In [ ]:
options(jupyter.rich_display = FALSE)

# Let's build a simple expert system for automated trading

**by Serhat Çevikel**

This suit of exercises is here to demonstrate that you can build rather complicated simulation and optimization models, using the tools you have learnt so far.

This will get complicated progressively. You can solve easily until the line chart (Exercise 3). Try to follow and understand, when you cannot solve yourself anymore.

We will use the built-in EuStockMarkets dataset

In [ ]:
EuStockMarkets

In [ ]:
dim(EuStockMarkets)

We have 1860 days of 4 indices

In [ ]:
str(EuStockMarkets)

Let's first get the annualized buy and hold returns:

In [ ]:
buyandhold <- (EuStockMarkets[nrow(EuStockMarkets),] / EuStockMarkets[1,])^
(365 / (nrow(EuStockMarkets)-1)) -1

In [ ]:
buyandhold

So DAX index returned a CAGR of 26.9% while SMI did 34.8%, CAC did 17.3% and FTSE did 17.1%

Can we beat the market? On annualized terms of course (meaning, we take into account the number of days we hold the buy position and annualize the return during this period)

## Simple moving average calculation

**EXERCISE 1:**

Write a function mova, that takes a vector x of numeric values and an n value of days.

The function will return the moving average of last n days of each value

You can make use of cumsum() function

So
```R
mova(1:10, 5)
```

will return

```R
[1] NA NA NA NA NA  3  4  5  6  7  8
```


In [ ]:
mova <- function(x, n = 5)
{
    x2 <- c(0, cumsum(x)) # cumulative sum, initialized with 0
    lagged <- seq_along(x2) - n # indices of n days before
    lagged[lagged < 1] <- NA # zero and negative indices replaced with NA
    movx <- (x2 - x2[lagged]) / n
    movx[-1]
}

In [ ]:
mova(1:10, 5)

## MACD

**EXERCISE 2:**

MACD index (moving averages convergence-divergence) is a major technical indicator in technical analysis of stock prices.

We take two moving averages of days n1 and n2 (n1 is shorter). MACD is the difference between those 2 series. We will change the definition a little bit and divide the shorter MA to longer MA

The function macd() will take three parameters:
- Numeric vector x
- Shorter number of days n1
- Longer number of days n2

Use function mova() inside macd()

So

```R
macd(EuStockMarkets[1:20,1], 2, 10)
```

will return:

```R
 [1]        NA        NA        NA        NA        NA        NA        NA
 [8]        NA        NA        NA 1.0095883 1.0122052 1.0083560 1.0014211
[15] 0.9962276 0.9942365 0.9950796 0.9972221 0.9964066 0.9920589 0.9895662
```

In [ ]:
macd <- function(x, n1 = 10, n2 = 100)
{
    mov1 <- mova(x, n1)
    mov2 <- mova(x, n2)
    mov1 / mov2
}

In [ ]:
macd(EuStockMarkets[1:20,1], 2, 10)

**EXERCISE 3:**

Now in two vertically stacked line plots:
- Show the first 1000 days of original DAX series
- Superimpose the 100 days moving average in red using lines() function
- Superimpose the 200 days moving average in blue using lines() function

- In the second plot, show the MACD for the same parameters
- Superimpose the horizontal line at 1

In [ ]:
plot(EuStockMarkets[1:1000,1], col = "black", type = "l")
lines(mova(EuStockMarkets[1:1000,1], 100), col = "red", type = "l")
lines(mova(EuStockMarkets[1:1000,1], 200), col = "blue", type = "l")

plot(macd(EuStockMarkets[1:1000,1], 100, 200), type = "l")
lines(rep(1, 1000))

## Buy or sell?

**EXERCISE 4:**

Now the hard part comes:
* Create a function macd_signal that takes four arguments: x vector n1 short days, n2 short days and a threshold value thres (default 1)
* The function will return a series of 0, -1, 1 values for which 0 means no signal, -1 means MACD line cut down the threshold value (a sell signal), 1 means MACD line cut up the threshold value (a buy signal)
* Note that first sell signal cannot be before the first buy signal (short selling not allowed). And on the last day there cannot be a buy signal (but there can be a sell signal)

macd_signal() function can make use of macd() function

So

```R
signal1 <- macd_signal(EuStockMarkets[1:50,2], 2, 10, 1)
```

will return

```R
 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
[26]  0  0 -1  0  0  1  0 -1  1  0  0 -1  0  0  0  1  0  0  0  0  0  0  0  0 -1
[51]  0
```


In [ ]:
macd_signal <- function(x, n1 = 10, n2 = 100, thres = 1)
{
    # calculate macd values
    macdx <- macd(x, n1, n2)
    
    # T for a buy signal
    buys <- c(NA, macdx[-length(macdx)] < thres & macdx[-1] >= thres)

    # T for a sell signal
    sells <- c(NA, macdx[-length(macdx)] >= thres & macdx[-1] < thres)

    # 0, -1, 1 vector of signals
    signals <- integer(length(x))
    signals[buys] <- 1
    signals[sells] <- -1
    
    # First sell signal predating first buy signal should be deleted
    first_sig <- match(c(-1,1), signals)
    if(first_sig[1] < first_sig[2]) signals[first_sig[1]] <- 0
    
    # NA's and a buy signal on the last day are treated
    signals[is.na(signals)] <- 0
    signals[length(signals)] <- min(0, signals[length(signals)])
    signals
}

In [ ]:
signal1 <- macd_signal(EuStockMarkets[1:50,2], 2, 10, 1)
signal1

## Simulate the signals

**EXERCISE 5:**

And a harder part:

Write a function simulate_signal that takes a vector of stock values and a vector of signals (of 0, -1, 1)
- We will calculate the cumulative return of stocks on the days starting from 1 signals and ending on -1 signals. We will annualize the returns 
- When no signals are created, the function should return 0 not NA

You can make use of cumsum() function

So:

```R
signal1 <- macd_signal(EuStockMarkets[,2], 2, 10, 1)
simulate_signal(EuStockMarkets[,2], signal1)
```

will return
```R
[1] 0.3558455
```

In [ ]:
simulate_signal <- function(x, signal1)
{
    buydays <- cumsum(signal1) # 0's are days with no posision and 1's are days with buy position
    buydays[length(buydays)] <- 0 # last day should be deleted not to have out-of-script error
    sigdays <- which(buydays == 1) ## indices of days with buy position
    totaldays <- sum(buydays) # number of days with buy position
    sigdays_1 <- sigdays + 1 # returns are calculated with one day lag
    returns <- x[sigdays_1] / x[sigdays] # calculate a vector of daily returns + 1
    total_ret <- prod(returns)^(365/totaldays) # get the product of returns + 1 and annualize
    ret <- total_ret - 1 # get the return
    if(is.na(ret)) 0 else ret # treat for NA
}

In [ ]:
signal1 <- macd_signal(EuStockMarkets[,2], 2, 10, 1)
simulate_signal(EuStockMarkets[,2], signal1)

**EXERCISE 6:**

Write a function macd_simulate() that takes x, n1, n2, thres, combines macd_signal and simulate_signal functions to yield the annualized return on the stock:

So:
```R
macd_simulate(EuStockMarkets[,1], n1 = 10, n2 = 100, thres = 1)
```
will yield:
```R
[1] 0.2721458
```


In [ ]:
macd_simulate <- function(x, n1 = 10, n2 = 100, thres = 1)
{
    signalx <- macd_signal(x, n1, n2, thres)
    simulate_signal(x, signalx)
}

In [ ]:
macd_simulate(EuStockMarkets[,1], n1 = 10, n2 = 100, thres = 1)

Let's apply the macd_simulate function on the column margin of EuStockMarkets data with selected parameters and compare with buyandhold:

In [ ]:
apply(EuStockMarkets, 2, macd_simulate, 10, 100, 1)

In [ ]:
buyandhold

## Optimize the parameters and beat the market!

**EXERCISE 7:**

Now let's optimize!

Create all the combinations of n1 (2:100) and n2 (3:200) values where n1 < n2 using expand.grid

Run the simulation for all those combinations

Which one to select? We don't want excessive return on one stock to dominate weak returns on other stocks. So we take the product of (1+return) for all four indices and find the combination that yields the maximum: 

In [ ]:
grid1 <- expand.grid(2:100, 3:200)

In [ ]:
grid1

In [ ]:
grid1 <- with(grid1, grid1[Var1 < Var2,])
grid1

In [ ]:
dim(grid1)

In [ ]:
simuls <- t(apply(grid1, 1, function(x) c(x,
                                          apply(EuStockMarkets,
                                                2,
                                                macd_simulate,
                                                x[1],
                                                x[2],
                                                1))))

In [ ]:
simuls

In [ ]:
buyandhold

In [ ]:
simuls2 <- t(apply(simuls, 1, function(x) c(x, prod(x[3:6] + 1))))
simuls2

In [ ]:
simuls2[order(-simuls2[,7]),]

Let's assume we divide our portfolio evenly across four indices.

The annualized return of buy and hold strategy is:

In [ ]:
mean(buyandhold)

The annualized return for the best n1 and n2 values is:

In [ ]:
mean(simuls2["1",3:6])

So we beat the market in annualized terms. But the implicit assumption is that, for the days during which there is no buy position, we have to invest our proceeds with the same annualized return! So we have to have a larger set of alternative securities...

What if we sort according to average returns?

In [ ]:
simuls3 <- t(apply(simuls, 1, function(x) c(x, mean(x[3:6]))))
simuls3[order(-simuls3[,7]),]

We get the same result for the best combination!